In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [2]:
def createGraph(Nodes,i):
    graphType = randint(0, 6, 1)
    while True:
        s = np.array(sample(range(1, 10000), 1))
        if graphType==0:
            G = pg.graphs.BarabasiAlbert(N = Nodes,seed = s)
            typeName = 'BarabasiAlbert'
        elif graphType == 1:
            G = pg.graphs.RandomRegular(N = Nodes,seed = s)
            typeName = 'RandomRegular'
        elif graphType == 2:
            G = pg.graphs.DavidSensorNet(N = Nodes,seed = s)
            typeName = 'DavidSensorNet'
        elif graphType == 3:
            G = pg.graphs.Community(N = Nodes,Nc = 3,seed = s)
            typeName = 'Community'
        elif graphType == 4:
            G = pg.graphs.ErdosRenyi(N = Nodes,p = 0.1,seed = s)
            typeName = 'ErdosRenyi'
        else:
            G = pg.graphs.Sensor(N=Nodes,seed = s)
            typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
            
    print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [3]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [4]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [5]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):
    A = np.tile(np.identity(N), (1, k))
    
    H = np.zeros((N*k,N*k))
    for i in range(k):
        g = graphs[i].L
        H[i*N:(i+1)*N,i*N:(i+1)*N] = g.toarray()
        
        
    X = cp.Variable((N*k,1))
    objective = cp.Minimize(cp.quad_form(X, H)) 
    
    constraints = [ A @ X == mainSignal]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=True)
    
    temp = X.value
    X_hat = temp.reshape(N,k, order='F')
    
    return X_hat

In [6]:
def makeMeanZero(signals,k):
    for i in range(k):
            signals[:,i] = signals[:,i] - np.mean(signals[:,i])
            
    return signals

In [7]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [8]:
N = 450    # number of nodes
k = 5      # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# experiment = ',itera,' ########################################')
    # create graph and graph signals
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # make mean of the estimated signals zero
    estimated_signals = makeMeanZero(estimated_signals,k)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

############################################# experiment =  1  ########################################


2021-12-03 14:59:56,918:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


ErdosRenyi
[26.88731118 27.59783794 28.33788505]


2021-12-03 14:59:57,443:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 14:59:57,905:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 14:59:58,431:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 14:59:59,028:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528
2021-12-03 14:59:59,717:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


Community
[0.08673535 0.31674044 0.53815972]
Sensor
[0.06809214 0.07604609 0.13376286]
BarabasiAlbert
[0.00658864 0.0088679  0.01365717]
BarabasiAlbert
[0.00358498 0.00655854 0.00893554]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 22910
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res 

...

Community
[0.40835554 0.54063676 0.5548798 ]


2021-12-03 15:27:54,101:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


BarabasiAlbert
[0.00552521 0.0097957  0.01021744]


2021-12-03 15:27:54,744:[INFO](pygsp.graphs.community.__init__): Constructed using eps-NN with eps = 3.2567778121631528


Community
[0.15878746 0.34766957 0.4053732 ]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 19020
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   1.98e+00   2.17e+03   1.00e-01   5.38e-01s
  50   1.0326e+01   3.46e-09   2.92e-08   1.00e-01   7.46e-0